#### Time Series Anomaly Exercises

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

Hint: You will need to explore the database and significantly adjust your acquisition step (yay SQL!)

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

In [7]:
# let's pull the dataset from the sql server
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
SELECT *
FROM logs
LEFT JOIN cohorts on cohorts.id = logs.cohort_id;
'''
df = pd.read_sql(query, url)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       object 
 14  program_id  847330 non-null  float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [19]:
# identify the number of nulls by column
df.isna().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

In [20]:
# let's clean up the dataset

# missing entirety of columns
df.drop('deleted_at', inplace = True, axis = 1)

# we need to identify which row(student) belongs to what cohort, if these don't have
# the cohort id, then we can't identify them.
df = df.dropna()

# change date type columns to the respective type 
df.date = pd.to_datetime(df.date)
df.time = pd.to_datetime(df.time)

# set the index
df = df.set_index(df.date)
df.rename(columns = {'path':'endpoint', 'ip':'source_ip'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847329 entries, 2018-01-26 to 2021-04-21
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        847329 non-null  datetime64[ns]
 1   time        847329 non-null  datetime64[ns]
 2   endpoint    847329 non-null  object        
 3   user_id     847329 non-null  int64         
 4   cohort_id   847329 non-null  float64       
 5   source_ip   847329 non-null  object        
 6   id          847329 non-null  float64       
 7   name        847329 non-null  object        
 8   slack       847329 non-null  object        
 9   start_date  847329 non-null  object        
 10  end_date    847329 non-null  object        
 11  created_at  847329 non-null  object        
 12  updated_at  847329 non-null  object        
 13  program_id  847329 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(8)
memory usage: 97.0+ MB


1. Label students by the program they are in.

In [21]:
df.name.unique()

array(['Hampton', 'Teddy', 'Sequoia', 'Arches', 'Niagara', 'Pinnacles',
       'Quincy', 'Kings', 'Lassen', 'Mammoth', 'Glacier', 'Denali',
       'Joshua', 'Olympic', 'Ulysses', 'Badlands', 'Apollo', 'Ike',
       'Voyageurs', 'Wrangell', 'Xanadu', 'Franklin', 'Yosemite', 'Staff',
       'Zion', 'Andromeda', 'Betelgeuse', 'Ceres', 'Bayes', 'Deimos',
       'Europa', 'Fortuna', 'Curie', 'Apex', 'Ganymede', 'Everglades',
       'Hyperion', 'Darden', 'Bash', 'Jupiter', 'Kalypso', 'Luna',
       'Easley', 'Marco', 'Neptune', 'Florence', 'Oberon'], dtype=object)

In [31]:
df[['user_id','name']].value_counts()

user_id  name   
11       Staff      15178
64       Staff      12530
53       Lassen      8197
495      Deimos      6451
428      Staff       5819
                    ...  
212      Arches         1
879      Marco          1
952      Olympic        1
163      Quincy         1
165      Niagara        1
Length: 952, dtype: int64

2. Is it possible to identify the user_id of a staff member?

In [37]:
df[df['name'] == 'Staff'].user_id.unique()

array([ 53, 314,  40,  64,  11, 211,   1, 312, 146, 248, 370, 397, 404,
       257, 428, 461,  37, 514, 539, 545, 546, 572, 315,  41, 592, 618,
       620, 521, 652, 502, 653, 480, 738, 742, 745, 813, 430, 816, 581,
       854, 855, 744, 893, 148, 894, 513, 630, 308, 951, 953, 980])

3. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

4. Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

5. Plot the access activity of these students.

BONUS: Identify users who are viewing both the web dev and data science curriculum